# 0. MÓDULOS USADOS

In [1]:
use strict;
use warnings;
use Data::Dump qw(dump);
use d2l;
use Chart::Plotly::Trace::Bar;
use Chart::Plotly::Plot;

IPerl->load_plugin('Chart::Plotly');

# 1. LECTURA Y PREPROCESAMIENTO PARA LA ALINEACION DE LOS CORPUS

## 1.1 Lectura de archivos
Con la función leer_archivo se leerá cada uno de los corpus (OCR y REF) para el desarrollo del proyecto.
El uso de shift es para recibir parámetros en una función para extraer y eliminar el primer elemento de la lista de argumentos (necesario cuando se procesa parámetros de manera secuencial, obteniendolos uno por uno)
Encoding(UTF-8) sirve para garantizar que los caracteres se lean correctamente si el archivo contiene caracteres no ASCII.
Retorna un map que devuelve una nueva lista con los cambios pedidos por los regex.
1. Regex elimina espacios en blanco
2. Luego se aplica un regex para cambiar los caracteres no alfanuméricos por espacios en blanco con el fin de formar palabras solo con caracteres alfanuméricos
3. Regex que elimina espacios al principio y al final de cada linea
4. Conversión de caracteres a minúsculas
5. grep filtra lineas vacias o que contengan solo espacios en blanco

### 1.1.1 Lectura data original

In [2]:
sub leer_archivo {
    my $archivo = shift;

    open my $fh, '<:encoding(UTF-8)', $archivo or die "ERROR: no se pudo abrir el archivo '$archivo'\n";
    my @lineas;
    
    while (my $linea = <$fh>) {
        chomp $linea;
        $linea =~ s/\s//g;
        $linea =~ s/^\W+|\W+$|\W+/ /g;
        $linea =~ s/^\s+|\s+$//g;
        push @lineas, lc($linea) unless $linea =~ /^\s+$/;
    }
    close $fh;
    
    return \@lineas;
}

## 1.2 Asignación de variables con el contenido de cada corpus ya preprocesado

In [3]:
my $tiempo = new d2l::Timer();

my $corpus_OCR = leer_archivo("../../TesisModelo1/0_Corpus/0_Testeo/dev.tok.fr");
my $corpus_REF = leer_archivo("../../TesisModelo1/0_Corpus/0_Testeo/dev.tok.en");

print "Duracion: ", $tiempo->stop(), "s.\n";

Duracion: 1.08344507217407s.


1

## 1.3 Revisión y segundo preprocesamiento de los corpus

### 1.3.1 Comprobación de la cadena más larga en los dos corpus

In [4]:
sub oracion_mas_larga {
  my $corpus = shift;

  my $oracion_mas_larga = "";
  my $longitud_maxima = 0;

  foreach my $oracion (@{$corpus}) {
    my @palabras = split(' ', $oracion);
    my $longitud_oracion = scalar @palabras;

    if ($longitud_oracion > $longitud_maxima) {
      $longitud_maxima = $longitud_oracion;
      $oracion_mas_larga = $oracion;
    }
  }

  return $oracion_mas_larga, $longitud_maxima;
}

In [5]:
my ($oracion_mas_larga_OCR, $longitud_mas_larga_OCR) = oracion_mas_larga($corpus_OCR);
my ($oracion_mas_larga_REF, $longitud_mas_larga_REF) = oracion_mas_larga($corpus_REF);

print "La oración más larga en el archivo OCR es:\n\n$oracion_mas_larga_OCR\n\n";
print "Número de palabras: $longitud_mas_larga_OCR\n\n\n";

print "La oración más larga en el archivo REF es:\n\n$oracion_mas_larga_REF\n\n";
print "Número de palabras: $longitud_mas_larga_REF\n\n";

La oración más larga en el archivo OCR es:

i do most of what i do through a macbook pro 15 macbook pro the last of the models with beveled keys i can t replace it because i can t type on the flat keys i ve got a typing speed of around 120 words per minute on a beveled keyboard but the chicklet keys on newer macs force my pinkies out too far the imac screens are too high and shiny for me so i m pretty well locked into the laptop for any intensive work i use a 20 imac 11 2 for skype movie watching and audio video editing

Número de palabras: 103


La oración más larga en el archivo REF es:

i do most of what i do through a macbook pro 15 macbook pro the last of the models with beveled keys i can t replace it because i can t type on the flat keys i ve got a typing speed of around 120 words per minute on a beveled keyboard but the chiclet keys on newer macs force my pinkies out too far the imac screens are too high and shiny for me so i m pretty well locked into the laptop for any intensi

1

### 1.3.2 Cálculo de la frecuencia de palabras por fila

In [6]:
sub frecuencia_palabras_por_fila {
    my ($corpus) = @_;
    my %contador_frecuencia_palabras;
    undef %contador_frecuencia_palabras;

    foreach my $oracion (@{$corpus}) {
        my @palabras = split(' ', $oracion);
        my $oracion_len = scalar @palabras;

        $contador_frecuencia_palabras{$oracion_len}++;
    }

    return %contador_frecuencia_palabras;
}

### 1.3.3 Histograma de frecuencias y número de palabras

In [7]:
sub histograma {
    my ($titulo, $len, $frecuencias, $nombre) = @_;

    my $bar_trace = Chart::Plotly::Trace::Bar->new(
        x    => $len,
        y    => $frecuencias,
        name => $nombre,
    );

    my $plot = Chart::Plotly::Plot->new(
        traces => [$bar_trace],
        layout => {
            title => {
                text => $titulo,
                font => {
                    family => "Arial",
                    size   => 30
                }
            },
            xaxis => { title => 'Número de palabras' },
            yaxis => { title => 'Frecuencia' },
        },
    );

    return $plot;
}

In [8]:
my %contador_frecuencia_palabras_REF = frecuencia_palabras_por_fila($corpus_REF);
my %contador_frecuencia_palabras_OCR = frecuencia_palabras_por_fila($corpus_OCR);

my @len_REF = sort { $a <=> $b } keys %contador_frecuencia_palabras_REF;
my @frecuencias_REF = @contador_frecuencia_palabras_REF{@len_REF};

my @len_OCR = sort { $a <=> $b } keys %contador_frecuencia_palabras_OCR;
my @frecuencias_OCR = @contador_frecuencia_palabras_OCR{@len_OCR};

192585385756004975075005696587997998837306604673642552242011841631371009210677827559665464695844595040334144484343395048293125403334252320252516281818211526131110131320191710127412966485953141111211

In [9]:
my $plot_OCR = histograma(
    'Distribución de frecuencias de número de palabras de OCR',
    \@len_OCR,
    \@frecuencias_OCR,
    'Frecuencia OCR'
);

In [10]:
my $plot_REF = histograma(
    'Distribución de frecuencias de número de palabras de REF',
    \@len_REF,
    \@frecuencias_REF,
    'Frecuencia REF'
);

### 1.3.4 Comprobación de filas vacías, filas que esten vacias en un archivo y que en el otro tenga una sola palabra, filas en la misma posicion de los dos archivos que solo contengan una palabra y caracteres individuales en los archivos

In [11]:
sub numero_palabras {
    my ($corpus, $target_len) = @_;
    my $pos = 1;
    my $filas_encontradas = 0;

    foreach my $oracion (@{$corpus}) {
        my @palabras = split(' ', $oracion);
        my $oracion_len = scalar @palabras;

        if ($oracion_len == $target_len) {
            print "Fila con $target_len tokens encontrada en la posición $pos\n";
            $filas_encontradas++;
        }

        $pos++;
    }

    return $filas_encontradas;
}

In [12]:
my $target_len_OCR = 1;
my $filas_encontradas_OCR = numero_palabras($corpus_OCR, $target_len_OCR);
print "Número de filas encontradas en OCR: $filas_encontradas_OCR\n";

Fila con 1 tokens encontrada en la posición 14
Fila con 1 tokens encontrada en la posición 59
Fila con 1 tokens encontrada en la posición 139
Fila con 1 tokens encontrada en la posición 150
Fila con 1 tokens encontrada en la posición 188
Fila con 1 tokens encontrada en la posición 220
Fila con 1 tokens encontrada en la posición 225
Fila con 1 tokens encontrada en la posición 345
Fila con 1 tokens encontrada en la posición 346
Fila con 1 tokens encontrada en la posición 353
Fila con 1 tokens encontrada en la posición 357
Fila con 1 tokens encontrada en la posición 439
Fila con 1 tokens encontrada en la posición 444
Fila con 1 tokens encontrada en la posición 659
Fila con 1 tokens encontrada en la posición 675
Fila con 1 tokens encontrada en la posición 678
Fila con 1 tokens encontrada en la posición 735
Fila con 1 tokens encontrada en la posición 760
Fila con 1 tokens encontrada en la posición 945
Fila con 1 tokens encontrada en la posición 1143
Fila con 1 tokens encontrada en la posici

1

In [13]:
my $target_len_REF = 1;
my $filas_encontradas_REF = numero_palabras($corpus_REF, $target_len_REF);
print "Número de filas encontradas en REF: $filas_encontradas_REF\n";

Fila con 1 tokens encontrada en la posición 14
Fila con 1 tokens encontrada en la posición 59
Fila con 1 tokens encontrada en la posición 139
Fila con 1 tokens encontrada en la posición 150
Fila con 1 tokens encontrada en la posición 188
Fila con 1 tokens encontrada en la posición 220
Fila con 1 tokens encontrada en la posición 225
Fila con 1 tokens encontrada en la posición 310
Fila con 1 tokens encontrada en la posición 345
Fila con 1 tokens encontrada en la posición 346
Fila con 1 tokens encontrada en la posición 353
Fila con 1 tokens encontrada en la posición 357
Fila con 1 tokens encontrada en la posición 439
Fila con 1 tokens encontrada en la posición 444
Fila con 1 tokens encontrada en la posición 659
Fila con 1 tokens encontrada en la posición 675
Fila con 1 tokens encontrada en la posición 678
Fila con 1 tokens encontrada en la posición 735
Fila con 1 tokens encontrada en la posición 760
Fila con 1 tokens encontrada en la posición 945
Fila con 1 tokens encontrada en la posició

1

In [14]:
sub caracteres_unicos {
    my ($corpus) = @_;

    my @matching_lineas;

    for (my $i = 0; $i < scalar @$corpus; $i++) {
        my $linea = $corpus->[$i];
        if ($linea =~ /^(\w\s\w\s?\w\s?)+\w$/) {
            push @matching_lineas, { linea_numero => $i + 1, linea_contenido => $linea };
        }
    }

    return @matching_lineas;
}

In [15]:
my @caracteres_unicos_OCR = caracteres_unicos($corpus_OCR);

foreach my $linea (@caracteres_unicos_OCR) {
    my $linea_numero = $linea->{linea_numero};
    my $linea_contenido = $linea->{linea_contenido};
    print "Se encontró una fila con caracteres individuales separados por espacios en la fila $linea_numero: $linea_contenido\n";
}

Se encontró una fila con caracteres individuales separados por espacios en la fila 1018: t end
Se encontró una fila con caracteres individuales separados por espacios en la fila 1279: 5 log uut
Se encontró una fila con caracteres individuales separados por espacios en la fila 2844: k h h i
Se encontró una fila con caracteres individuales separados por espacios en la fila 3853: o i a i j b j


In [16]:
my @caracteres_unicos_REF = caracteres_unicos($corpus_REF);

foreach my $linea (@caracteres_unicos_REF) {
    my $linea_numero = $linea->{linea_numero};
    my $linea_contenido = $linea->{linea_contenido};
    print "Se encontró una fila con caracteres individuales separados por espacios en la fila $linea_numero: $linea_contenido\n";
}

Se encontró una fila con caracteres individuales separados por espacios en la fila 1279: 5 log out
Se encontró una fila con caracteres individuales separados por espacios en la fila 2844: j h h i
Se encontró una fila con caracteres individuales separados por espacios en la fila 3853: o i a i j x j
Se encontró una fila con caracteres individuales separados por espacios en la fila 5385: n nak n n i
Se encontró una fila con caracteres individuales separados por espacios en la fila 12100: f 0 01


### 1.3.4 Eliminación de filas vacias

In [17]:
sub comparar_filtrar {
    my ($corpus_OCR, $corpus_REF) = @_;

    my @corpus_OCR = @{$corpus_OCR};
    my @corpus_REF = @{$corpus_REF};

    my @filtrar_corpus_OCR;
    my @filtrar_corpus_REF;

    for my $i (0 .. $#corpus_OCR) {
        my $linea_OCR = $corpus_OCR[$i];
        my $linea_REF = $corpus_REF[$i];

        chomp $linea_OCR;
        chomp $linea_REF;

        next if $linea_OCR =~ /^(\w\s\w\s?\w\s?)+\w$/ || $linea_REF =~ /^(\w\s\w\s?\w\s?)+\w$/;
        next if $linea_OCR =~ /^\s*$/ && $linea_REF =~ /^\s*$/;
        next if $linea_OCR =~ /^\w+$/ && $linea_REF =~ /^\w+$/;

        if ($linea_OCR =~ /^\s*$/ && $linea_REF =~ /^\w+$/) {
            next;
        }

        if ($linea_OCR =~ /^\w+$/ && $linea_REF =~ /^\s*$/) {
            next;
        }

        push @filtrar_corpus_OCR, $linea_OCR;
        push @filtrar_corpus_REF, $linea_REF;
    }

    return (\@filtrar_corpus_OCR, \@filtrar_corpus_REF);
}

In [18]:
($corpus_OCR, $corpus_REF) = comparar_filtrar($corpus_OCR, $corpus_REF);

ARRAY(0xcd77ef8)ARRAY(0xcd77e80)

### 1.3.5 Eliminar oraciones que tengan una sola palabra

In [19]:
sub contador_palabras {
    my ($corpus_OCR, $corpus_REF) = @_;

    my @filtrar_OCR;
    my @filtrar_REF;

    for (my $i = 0; $i < scalar @$corpus_OCR; $i++) {
        my $linea_OCR = $corpus_OCR->[$i];
        my $linea_REF = $corpus_REF->[$i];

        my @palabras_OCR = split /\s+/, $linea_OCR;
        my @palabras_REF = split /\s+/, $linea_REF;

        if ((scalar @palabras_OCR<2 || scalar @palabras_REF<2)) {
            next;
        }

        push @filtrar_OCR, $linea_OCR;
        push @filtrar_REF, $linea_REF;
    }

    return (\@filtrar_OCR, \@filtrar_REF);
}


In [20]:
($corpus_OCR, $corpus_REF) = contador_palabras($corpus_OCR, $corpus_REF);

ARRAY(0xc9636e8)ARRAY(0xc963700)

In [21]:
my %contador_frecuencia_palabras_REF = frecuencia_palabras_por_fila($corpus_REF);
my %contador_frecuencia_palabras_OCR = frecuencia_palabras_por_fila($corpus_OCR);

my @len_REF = sort { $a <=> $b } keys %contador_frecuencia_palabras_REF;
my @frecuencias_REF = @contador_frecuencia_palabras_REF{@len_REF};

my @len_OCR = sort { $a <=> $b } keys %contador_frecuencia_palabras_OCR;
my @frecuencias_OCR = @contador_frecuencia_palabras_OCR{@len_OCR};

5305735984975074995696587997998837306604673642552242011841631371009210677827559665464695844595040334144484343395048293125403334252320252516281818211526131110131320191710127412966485953141111211

In [22]:
my $plot_OCR = histograma(
    'Distribución de frecuencias de número de palabras de OCR',
    \@len_OCR,
    \@frecuencias_OCR,
    'Frecuencia OCR'
);

In [23]:
my $plot_REF = histograma(
    'Distribución de frecuencias de número de palabras de REF',
    \@len_REF,
    \@frecuencias_REF,
    'Frecuencia REF'
);

In [24]:
# for my $let (0 .. 100) {
#     print dump($corpus_OCR->[$let])."\n";
# }

### 1.3.6 Contar el número de caracteres por palabra

In [25]:
sub contar_caracteres_por_palabra {
    my ($corpus, $top_n) = @_;
    my %palabra_info;

    for my $fila (0 .. $#{$corpus}) {
        my @palabras = split(" ", $corpus->[$fila]);
        for my $palabra (@palabras) {
            $palabra =~ s/^\s+|\s+$//g;  # Elimina espacios en blanco al principio y al final
            my $longitud = length($palabra);
            push @{$palabra_info{$palabra}}, [$longitud, $fila + 1];
        }
    }

    # Ordenar el resultado por longitud de caracteres en orden descendente
    my @resultado_ordenado = sort { length($b) <=> length($a) } keys %palabra_info;

    # Mostrar las palabras más largas
    for my $i (0 .. $top_n - 1) {
        last if $i >= @resultado_ordenado;
        my $palabra = $resultado_ordenado[$i];
        my $info = $palabra_info{$palabra};
        my $fila_maxima = (sort { $b->[0] <=> $a->[0] } @$info)[0];
        print "Palabra: $palabra, Longitud: $fila_maxima->[0], Fila: $fila_maxima->[1]\n";
    }
}

In [26]:
contar_caracteres_por_palabra($corpus_OCR, 10);

Palabra: d73jds8f73jkr9d8sufv2br8sd9fy92nr9f80u23r97fhse, Longitud: 47, Fila: 2605
Palabra: cc52e06b490e0dc2482b345aa5d0d65fae931095, Longitud: 40, Fila: 9459
Palabra: 86013bc11247d2e58d04bf52ee5462a927003f58, Longitud: 40, Fila: 11528
Palabra: b9157ce933c79a4559d2aa9ff3372668cce48de7, Longitud: 40, Fila: 9450
Palabra: scrollstobottomonkeybordbeginsediting, Longitud: 37, Fila: 6974
Palabra: segmentedpagertabstripviewcontroller, Longitud: 36, Fila: 12059
Palabra: buttonbarpagertabstripviewcontroller, Longitud: 36, Fila: 12059
Palabra: rawreadunpackscoalsescedinputrecords, Longitud: 36, Fila: 7187
Palabra: getmodifierstatusresistancestacking, Longitud: 35, Fila: 9836
Palabra: cborprettyindicateindeterminelength, Longitud: 35, Fila: 5441


In [27]:
open(my $archivo_OCR, '>:encoding(UTF-8)', '../../TesisModelo1/0_Corpus/1_Testeo/corpus_OCR_preprocesada.txt') or die "No se puede abrir el archivo corpus_OCR_preprocesada.txt: $!";
foreach my $linea (@{$corpus_OCR}) {
    print $archivo_OCR $linea . "\n";
}
close($archivo_OCR);

open(my $archivo_REF, '>:encoding(UTF-8)', '../../TesisModelo1/0_Corpus/1_Testeo/corpus_REF_preprocesada.txt') or die "No se puede abrir el archivo corpus_REF_preprocesada.txt: $!";
foreach my $linea (@{$corpus_REF}) {
    print $archivo_REF $linea . "\n";
}
close($archivo_REF);

1